In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [127]:
dataset=pd.read_csv('Historical Product Demand.csv')

In [128]:
dataset.head(10)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
5,Product_0979,Whse_J,Category_028,2012/4/19,500
6,Product_0979,Whse_J,Category_028,2012/6/5,500
7,Product_0979,Whse_J,Category_028,2012/6/27,500
8,Product_0979,Whse_J,Category_028,2012/7/23,500
9,Product_0979,Whse_J,Category_028,2012/8/29,500


In [129]:
# 1 percent of data has missing date variable. Hence we can drop them
(dataset.isnull().sum().Date/dataset.shape[0])*100

1.0718355863910547

In [130]:
dataset=dataset.dropna()

In [131]:
dataset.isnull().sum()

Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
dtype: int64

In [133]:
dataset['Product_Code']=dataset['Product_Code'].map(lambda x: int(x.strip('Product_').strip(' ')))
dataset['Warehouse'] = dataset['Warehouse'].map(lambda x: x.strip('Whse_').strip(' '))
dataset['Product_Category'] = dataset['Product_Category'].map(lambda x: int(x.strip('Category').strip('_')))
dataset=pd.concat([dataset,pd.get_dummies(dataset.Warehouse,prefix='WH')],axis=1)
dataset["Order_Demand"]=dataset.Order_Demand.str.strip('()').apply(lambda x: int(x))
dataset["Year"]=dataset.Date.apply(lambda x: int(x[0:4]))
dataset["Month"]=dataset.Date.apply(lambda x: int(x[4:7].strip('/')))
dataset["Day"]=dataset.Date.apply(lambda x: int(x[7:].strip('/')))
import datetime
import math
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """
    # weekday from monday == 0 ---> sunday == 6
    last_day_of_week_of_month = dt.day + (7 - (1 + dt.weekday()))
    return int(math.ceil(last_day_of_week_of_month/7.0))
dataset["Week"]=dataset.Date.apply(lambda x: week_of_month(datetime.date(int(x[0:4]),int(x[4:7].strip('/')),int(x[7:].strip('/')))))
dataset=dataset.drop(['Warehouse','Date'],axis=1)

In [134]:
dataset.corr()

,Product_Code,Product_Category,Order_Demand,WH_A,WH_C,WH_J,WH_S,Year,Month,Day,Week
Product_Code,1.000000,0.146554,0.032076,0.013469,-0.034837,0.028750,-0.037283,0.016395,0.008216,0.001856,0.000539
Product_Category,0.146554,1.000000,0.054247,0.136577,0.016879,-0.179003,0.102117,-0.020446,0.004816,-0.001242,-0.002009
Order_Demand,0.032076,0.054247,1.000000,-0.053638,0.062907,-0.031655,0.071504,0.008249,-0.000642,-0.001986,-0.001361
WH_A,0.013469,0.136577,-0.053638,1.000000,-0.082278,-0.667459,-0.121567,-0.014334,0.012808,0.006613,0.003054
WH_C,-0.034837,0.016879,0.062907,-0.082278,1.000000,-0.345319,-0.062894,0.031556,0.012823,0.009537,0.009353
WH_J,0.028750,-0.179003,-0.031655,-0.667459,-0.345319,1.000000,-0.510212,0.030592,-0.015834,-0.014423,-0.012518
WH_S,-0.037283,0.102117,0.071504,-0.121567,-0.062894,-0.510212,1.000000,-0.053000,0.000097,0.007844,0.009357
Year,0.016395,-0.020446,0.008249,-0.014334,0.031556,0.030592,-0.053000,1.000000,-0.008263,-0.007045,-0.023793
Month,0.008216,0.004816,-0.000642,0.012808,0.012823,-0.015834,0.000097,-0.008263,1.000000,-0.034743,-0.050458
Day,0.001856,-0.001242,-0.001986,0.006613,0.009537,-0.014423,0.007844,-0.007045,-0.034743,1.000000,0.962114


In [135]:
dataset.head()

,Product_Code,Product_Category,Order_Demand,WH_A,WH_C,WH_J,WH_S,Year,Month,Day,Week
0,993,28,100,0,0,1,0,2012,7,27,5
1,979,28,500,0,0,1,0,2012,1,19,4
2,979,28,500,0,0,1,0,2012,2,3,1
3,979,28,500,0,0,1,0,2012,2,9,2
4,979,28,500,0,0,1,0,2012,3,2,1


In [142]:
y=dataset['Order_Demand'].values
X=dataset.drop('Order_Demand',axis=1).values
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_std=ss.fit_transform(X)
from sklearn.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(3)
X_Poly=pf.fit_transform(X_std)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [143]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_Poly,y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(xtrain,ytrain)
pred_train=linreg.predict(xtrain)
pred_test=linreg.predict(xtest)
from sklearn.metrics import mean_squared_error
lin_rmse_train = np.sqrt(mean_squared_error(ytrain,pred_train))
lin_rmse_test=np.sqrt(mean_squared_error(ytest,pred_test))
print("Train Error",lin_rmse_train,"Test Error",lin_rmse_test)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.plot(x['Date'].head(30), x['Order_Demand'].head(30))
plt.show()

In [111]:
# Train and Test Splitting is done
data_train=dataset[dataset['Year']<int('2016')]
#data_test=pd.concat([dataset, data_train]).drop_duplicates(keep=False)
data_test=dataset[dataset['Year']>=int('2016')]

#print('data train count',data_train.count(), 'datatest count', data_test.count())

print(data_train.count()+data_test.count())

Product_Code        1037336
Product_Category    1037336
Order_Demand        1037336
Year                1037336
Month               1037336
Day                 1037336
Week                1037336
Warehouse_A         1037336
Warehouse_C         1037336
Warehouse_J         1037336
Warehouse_S         1037336
dtype: int64


In [75]:
#data_train=pd.get_dummies(data_train, columns=['Warehouse'])
#data_test=pd.get_dummies(data_test, columns=['Warehouse'])

In [112]:
data_train.head(5)
data_test.head(5)

,Product_Code,Product_Category,Order_Demand,Year,Month,Day,Week,Warehouse_A,Warehouse_C,Warehouse_J,Warehouse_S
690943,1178,24,10,2016,1,28,4,1,0,0,0
699219,1502,19,100000,2016,1,4,1,0,1,0,0
768552,190,7,320,2016,1,6,1,1,0,0,0
768635,337,21,2,2016,1,6,1,1,0,0,0
768656,1053,24,10,2016,1,6,1,1,0,0,0


In [113]:
# segregating dependent and independent variable for train and test dataframes. 

X_train= data_train.drop('Order_Demand',axis=1).values
y_train= data_train['Order_Demand'].values

In [114]:
X_test= data_test.drop('Order_Demand',axis=1).values
y_test= data_test['Order_Demand'].values

In [115]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848638 entries, 0 to 1045241
Data columns (total 11 columns):
Product_Code        848638 non-null int64
Product_Category    848638 non-null int64
Order_Demand        848638 non-null int64
Year                848638 non-null int64
Month               848638 non-null int64
Day                 848638 non-null int64
Week                848638 non-null int64
Warehouse_A         848638 non-null uint8
Warehouse_C         848638 non-null uint8
Warehouse_J         848638 non-null uint8
Warehouse_S         848638 non-null uint8
dtypes: int64(7), uint8(4)
memory usage: 55.0 MB


In [116]:
#Feature Scaling 

from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train=sc.fit_transform(X_train)

C:\Users\deepti.thakur\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [117]:
X_train

array([[-0.20749903,  1.64891655, -1.36023829, ..., -0.19829901,
         0.61057416, -0.3261136 ],
       [-0.2318283 ,  1.64891655, -1.36023829, ..., -0.19829901,
         0.61057416, -0.3261136 ],
       [-0.2318283 ,  1.64891655, -1.36023829, ..., -0.19829901,
         0.61057416, -0.3261136 ],
       ...,
       [ 1.27484823, -1.12951412,  1.34643443, ..., -0.19829901,
         0.61057416, -0.3261136 ],
       [-1.82365724,  0.51228582,  1.34643443, ..., -0.19829901,
         0.61057416, -0.3261136 ],
       [-0.57417579, -1.76097563,  1.34643443, ..., -0.19829901,
         0.61057416, -0.3261136 ]])

In [118]:
from sklearn.linear_model import LinearRegression
linreg= LinearRegression()
linreg.fit(X_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [119]:
linreg.score(X_train,y_train)

0.014767829310487146